In [ ]:
!pip install qiskit
!pip install qiskit_aer
!pip install qiskit_ibm_runtime

In [1]:
# Deutsch–Jozsa Algorithm using Qiskit 2.x
# Compatible with Qiskit >= 2.0.0

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Options
from qiskit_aer import Aer
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# ---------- ORACLES ----------
def oracle_constant(qc, ancilla, value=0):
    """Constant oracle: f(x)=0 or f(x)=1"""
    if value == 1:
        qc.x(ancilla)


def oracle_balanced_parity(qc, inputs, ancilla):
    """Balanced oracle: f(x) = x0 XOR x1 XOR ... XOR xn"""
    for q in inputs:
        qc.cx(q, ancilla)




In [3]:
# ---------- DEUTSCH–JOZSA CIRCUIT ----------
def deutsch_jozsa_circuit(n, oracle_func, *oracle_args):
    """
    n: number of input qubits
    oracle_func: oracle function to modify the circuit
    oracle_args: extra arguments for oracle
    """
    qreg = QuantumRegister(n + 1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # Step 1: Initialize |0...0>|1>
    qc.x(ancilla)

    # Step 2: Apply Hadamard to all qubits
    qc.h(qreg)

    # Step 3: Oracle
    oracle_func(qc, *oracle_args)

    # Step 4: Apply Hadamard to input qubits
    for q in inputs:
        qc.h(q)

    # Step 5: Measure only input qubits
    qc.measure(inputs, creg)

    return qc


In [4]:
# ---------- EXECUTION ----------
def run_dj(qc):
    """Run Deutsch–Jozsa circuit on AerSimulator"""
    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()

    print("Measurement counts:", counts)
    plot_histogram(counts)
    plt.show()

    n = qc.num_clbits
    if counts.get("0" * n, 0) == 1024:
        print("✅ Function is CONSTANT")
    else:
        print("✅ Function is BALANCED")


In [5]:
# ---------- MAIN ----------
if __name__ == "__main__":
    n = 3  # number of input qubits

    print("\n=== Constant Oracle (f(x)=0) ===")
    qc_const = deutsch_jozsa_circuit(
        n, oracle_constant, n, 0
    )
    print(qc_const.draw(fold=-1))
    run_dj(qc_const)

    print("\n=== Balanced Oracle (Parity) ===")
    qc_balanced = deutsch_jozsa_circuit(
        n, oracle_balanced_parity, list(range(n)), n
    )
    print(qc_balanced.draw(fold=-1))
    run_dj(qc_balanced)



=== Constant Oracle (f(x)=0) ===
     ┌───┐┌───┐┌─┐      
q_0: ┤ H ├┤ H ├┤M├──────
     ├───┤├───┤└╥┘┌─┐   
q_1: ┤ H ├┤ H ├─╫─┤M├───
     ├───┤├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├┤ H ├─╫──╫─┤M├
     ├───┤├───┤ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├─╫──╫──╫─
     └───┘└───┘ ║  ║  ║ 
c: 3/═══════════╩══╩══╩═
                0  1  2 
Measurement counts: {'000': 1024}
✅ Function is CONSTANT

=== Balanced Oracle (Parity) ===
     ┌───┐          ┌───┐     ┌─┐           
q_0: ┤ H ├───────■──┤ H ├─────┤M├───────────
     ├───┤       │  └───┘┌───┐└╥┘     ┌─┐   
q_1: ┤ H ├───────┼────■──┤ H ├─╫──────┤M├───
     ├───┤       │    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_2: ┤ H ├───────┼────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_3: ┤ X ├┤ H ├┤ X ├┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘└───┘└───┘ ║       ║  ║ 
c: 3/══════════════════════════╩═══════╩══╩═
                               0       1  2 
Measurement counts: {'111': 1024}
✅ Function is BALANCED


In [6]:
"""
Task 1 — Modify the Oracle
Create your own balanced oracle function that flips the ancilla for half of all possible inputs (not just parity).
Hint: You can use cx and ccx gates creatively.
"""

def oracle_balanced_custom(qc, inputs, ancilla):
    """
    Balanced oracle for n=3: f(x) = (x0 AND x1) XOR x2
    This is just an example; it won't scale for n != 3.
    """
    # Implements (x0 AND x1)
    # Flips ancilla if inputs[0] and inputs[1] are both 1
    qc.ccx(inputs[0], inputs[1], ancilla)
    
    # Implements XOR x2
    # Flips ancilla if inputs[2] is 1
    qc.cx(inputs[2], ancilla)

if __name__ == "__main__":
    n = 3  # This oracle is hardcoded for n=3
    
    print("\n=== Custom Balanced Oracle (AND-XOR) ===")
    qc_balanced_custom = deutsch_jozsa_circuit(
        n, oracle_balanced_custom, list(range(n)), n
    )
    
    print(qc_balanced_custom.draw(fold=-1))
    run_dj(qc_balanced_custom)


=== Custom Balanced Oracle (AND-XOR) ===
     ┌───┐          ┌───┐     ┌─┐      
q_0: ┤ H ├───────■──┤ H ├─────┤M├──────
     ├───┤       │  ├───┤     └╥┘┌─┐   
q_1: ┤ H ├───────■──┤ H ├──────╫─┤M├───
     ├───┤       │  └───┘┌───┐ ║ └╥┘┌─┐
q_2: ┤ H ├───────┼────■──┤ H ├─╫──╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐└───┘ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├┤ X ├┤ X ├──────╫──╫──╫─
     └───┘└───┘└───┘└───┘      ║  ║  ║ 
c: 3/══════════════════════════╩══╩══╩═
                               0  1  2 
Measurement counts: {'100': 259, '110': 263, '111': 244, '101': 258}
✅ Function is BALANCED


In [7]:
"""
Task 2 — Change the Number of Input Qubits
Run the Deutsch–Jozsa algorithm with different numbers of input qubits (e.g., n = 2, 4, 5).
Observe how the circuit depth and output pattern change.
"""

def run_dj_for_n(n_inputs):
    print(f"   RUNNING FOR N = {n_inputs} INPUTS")
    
    # --- Test Constant Oracle ---
    print(f"\n--- Constant Oracle (f(x)=0) for n={n_inputs} ---")
    qc_const = deutsch_jozsa_circuit(
        n_inputs, oracle_constant, n_inputs, 0
    )
    print("Circuit depth:", qc_const.depth())
    print(qc_const.draw(fold=-1))
    run_dj(qc_const)

    # --- Test Balanced (Parity) Oracle ---
    print(f"\n--- Balanced Oracle (Parity) for n={n_inputs} ---")
    qc_balanced = deutsch_jozsa_circuit(
        n_inputs, oracle_balanced_parity, list(range(n_inputs)), n_inputs
    )
    print("Circuit depth:", qc_balanced.depth())
    print(qc_balanced.draw(fold=-1))
    run_dj(qc_balanced)

if __name__ == "__main__":
    run_dj_for_n(n_inputs=2)
    run_dj_for_n(n_inputs=4)
    run_dj_for_n(n_inputs=5)

   RUNNING FOR N = 2 INPUTS

--- Constant Oracle (f(x)=0) for n=2 ---
Circuit depth: 3
     ┌───┐┌───┐┌─┐   
q_0: ┤ H ├┤ H ├┤M├───
     ├───┤├───┤└╥┘┌─┐
q_1: ┤ H ├┤ H ├─╫─┤M├
     ├───┤├───┤ ║ └╥┘
q_2: ┤ X ├┤ H ├─╫──╫─
     └───┘└───┘ ║  ║ 
c: 2/═══════════╩══╩═
                0  1 
Measurement counts: {'00': 1024}
✅ Function is CONSTANT

--- Balanced Oracle (Parity) for n=2 ---
Circuit depth: 6
     ┌───┐          ┌───┐     ┌─┐   
q_0: ┤ H ├───────■──┤ H ├─────┤M├───
     ├───┤       │  └───┘┌───┐└╥┘┌─┐
q_1: ┤ H ├───────┼────■──┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤ X ├┤ X ├──────╫──╫─
     └───┘└───┘└───┘└───┘      ║  ║ 
c: 2/══════════════════════════╩══╩═
                               0  1 
Measurement counts: {'11': 1024}
✅ Function is BALANCED
   RUNNING FOR N = 4 INPUTS

--- Constant Oracle (f(x)=0) for n=4 ---
Circuit depth: 3
     ┌───┐┌───┐┌─┐         
q_0: ┤ H ├┤ H ├┤M├─────────
     ├───┤├───┤└╥┘┌─┐      
q_1: ┤ H ├┤ H ├─╫─┤M├──────
     ├───┤├─

In [8]:
"""
Task 3 — Add Noise Simulation
Simulate the same circuit using a noisy backend:

from qiskit_aer.noise import NoiseModel
and observe how measurement outcomes are affected.
"""

def get_simple_noise_model():
    noise_model = NoiseModel()
    
    # Add a 1% chance of a random error (depolarizing) 
    # after any CNOT or Hadamard gate.
    p_error = 0.01 
    depol_error = depolarizing_error(p_error, 1)
    depol_error_2q = depolarizing_error(p_error, 2)

    noise_model.add_all_qubit_quantum_error(depol_error, ['h'])
    noise_model.add_all_qubit_quantum_error(depol_error_2q, ['cx'])
    
    return noise_model

def run_dj_noisy(qc, noise_model):
    simulator = AerSimulator(noise_model=noise_model)
    
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()

    print("Noisy Measurement counts:", counts)
    plot_histogram(counts)
    plt.show()

    n = qc.num_clbits
    all_zeros = "0" * n
    correct_counts = counts.get(all_zeros, 0)
    
    print(f"Got correct answer '{all_zeros}' {correct_counts} out of 1024 times.")
    
    if correct_counts > 1024 * 0.8: # Check if it's correct >80% of the time
        print("✅ Function is LIKELY CONSTANT (despite noise)")
    else:
        print("✅ Function is LIKELY BALANCED (despite noise)")


if __name__ == "__main__":
    n = 3
    my_noise_model = get_simple_noise_model()

    print(f"  RUNNING N={n} WITH 1% NOISE")

    print("\n--- Noisy Constant Oracle ---")
    qc_const_noisy = deutsch_jozsa_circuit(
        n, oracle_constant, n, 0
    )
    run_dj_noisy(qc_const_noisy, my_noise_model)
    
    print("\n--- Noisy Balanced (Parity) Oracle ---")
    qc_balanced_noisy = deutsch_jozsa_circuit(
        n, oracle_balanced_parity, list(range(n)), n
    )
    run_dj_noisy(qc_balanced_noisy, my_noise_model)

  RUNNING N=3 WITH 1% NOISE

--- Noisy Constant Oracle ---
Noisy Measurement counts: {'000': 1024}
Got correct answer '000' 1024 out of 1024 times.
✅ Function is LIKELY CONSTANT (despite noise)

--- Noisy Balanced (Parity) Oracle ---
Noisy Measurement counts: {'111': 978, '001': 3, '011': 18, '000': 1, '110': 14, '101': 10}
Got correct answer '000' 1 out of 1024 times.
✅ Function is LIKELY BALANCED (despite noise)


In [ ]:
"""
Task 4 — Run on IBM Quantum Device
Modify the code to run on a real IBM Quantum backend using:

from qiskit_ibm_runtime import QiskitRuntimeService
Compare real hardware results with simulator outcomes.
"""

In [9]:
"""
Task 5 — Circuit Analysis
Print the unitary of your oracle using:

qc = QuantumCircuit(...)
oracle_func(qc, ...)
print(qc.to_gate().definition)
and explain how it implements the function ( f(x) ).
"""

def analyze_oracle(n, oracle_func, *oracle_args):
    
    # 1. Build a circuit with only the oracle
    qreg = QuantumRegister(n + 1)
    oracle_qc = QuantumCircuit(qreg)
    oracle_func(oracle_qc, *oracle_args)
    
    print(f"\n--- Analysis for: {oracle_func.__name__} ---")
    
    # --- Method 1: Print the gates ---
    print("\n[Method 1: Circuit Definition (Gates)]")
    print(oracle_qc.draw(fold=-1))
    
    # --- Method 2: Get the Unitary Matrix ---
    print("\n[Method 2: Unitary Matrix (Math Definition)]")
    sim = Aer.get_backend('unitary_simulator')
    
    t_oracle_qc = transpile(oracle_qc, sim) 
    
    job = sim.run(t_oracle_qc)
    result = job.result()
    
    unitary = result.get_unitary() 
    unitary_array = np.asarray(unitary) 

    print(f"Matrix shape: {unitary_array.shape}")
    np.set_printoptions(precision=1, suppress=True)
    print("Top-left 4x4 corner of the matrix:")
    print(unitary_array[:4, :4])

if __name__ == "__main__":
    n = 2 # Use n=2 (3 qubits total) so matrix is 8x8
    
    # Analyze the constant oracle
    analyze_oracle(n, oracle_constant, n, 0)
    
    # Analyze the balanced parity oracle
    analyze_oracle(n, oracle_balanced_parity, list(range(n)), n)


--- Analysis for: oracle_constant ---

[Method 1: Circuit Definition (Gates)]
      
q0_0: 
      
q0_1: 
      
q0_2: 
      

[Method 2: Unitary Matrix (Math Definition)]
Matrix shape: (8, 8)
Top-left 4x4 corner of the matrix:
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]

--- Analysis for: oracle_balanced_parity ---

[Method 1: Circuit Definition (Gates)]
                
q1_0: ──■───────
        │       
q1_1: ──┼────■──
      ┌─┴─┐┌─┴─┐
q1_2: ┤ X ├┤ X ├
      └───┘└───┘

[Method 2: Unitary Matrix (Math Definition)]
Matrix shape: (8, 8)
Top-left 4x4 corner of the matrix:
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]
